# Taking Derivatives of `MultibodyPlant` Computations w.r.t. Mass
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

우리는 간단한 `MultibodyPlant` (`MultibodyPlant_[float]`)을 만듭다. 이는 URDF / SDFormat 파일에서 구문 분석할 수 있음을 의미한다. 그런 다음 시스템을 `AutoDiffXd`로 변환하고 원하는 파라미터와 편도함수를 설정한다. 이 경우  
$\frac{\partial{\boldsymbol{f}}}{\partial{\boldsymbol{m}}}$를 원하며, 여기서 $\boldsymbol{f}$는 임의의 표현이다.

우리의 경우, $\boldsymbol{f}$를 기본/홈 구성에서 일반화된 힘으로 선택한다. 또한 이 경우에서 $\boldsymbol{m} = \left[ m_1, m_2 \right] \in \mathbb{R}_+^2$를 선택하여 독립적인 값에 대한 그래디언트를 어떻게 선택하는지 보여준다.

관련된 읽을꺼리:
- [Underactuated: System Identification](http://underactuated.csail.mit.edu/sysid.html) - at present, this only presents the symbolic approach for `MultibodyPlant`.
- [`Modeling Dynamical Systems`](./dynamical_systems.ipynb)
- [`Mathematical Program MultibodyPlant Tutorial`](./mathematical_program_multibody_plant.ipynb)

In [ ]:
import numpy as np

from pydrake.multibody.tree import SpatialInertia, UnitInertia
from pydrake.multibody.plant import MultibodyPlant_, MultibodyPlant
from pydrake.autodiffutils import AutoDiffXd

In [ ]:
plant = MultibodyPlant(time_step=0.0)
body1 = plant.AddRigidBody(
    "body1",
    M_BBo_B=SpatialInertia(
        mass=2.0,
        p_PScm_E=[0, 0, 0],
        # N.B. Rotational inertia is unimportant for calculations
        # in this notebook, and thus is arbitrarily chosen.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
body2 = plant.AddRigidBody(
    "body2",
    M_BBo_B=SpatialInertia(
        mass=0.5,
        p_PScm_E=[0, 0, 0],
        # N.B. Rotational inertia is unimportant for calculations
        # in this notebook, and thus is arbitrarily chosen.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
plant.Finalize()

plant_ad = plant.ToScalarType[AutoDiffXd]()
body1_ad = plant_ad.get_body(body1.index())
body2_ad = plant_ad.get_body(body2.index())
context_ad = plant_ad.CreateDefaultContext()

"특정 파라미터에 대한 그래디언트를 계산해야 하므로, 이제 해당 파라미터를 채워넣자.

순방향 자동 미분(forward-mode automatic differentiation)을 위해 원하는 독립 변수에 따라 그래디언트를 지정해야 한다. 이 경우 우리는 $m_1$ 와 $m_2$가 독립적으로 존재하기를 원하므로, 그것들의 그래디언트가 서로 다른 단위 벡터임을 보장한다. (이 작업을 위해 [`InitializeAutoDiff`](https://drake.mit.edu/pydrake/pydrake.autodiffutils.html#pydrake.autodiffutils.InitializeAutoDiff)를 사용할 수도 있지만, 여기서는 설명을 위해 "직접" 수행한다.)

원한다면 더 많은 파라미터 (예: 질량 중심(center-of-mass))를 설정할 수도 있지만, 간략히 하기 위해서 질량만 다루도록 하겠다.

이 경우 "자동 미분 가능한(autodiff-able)" 모든 다른 양 (상태, 다른 파라미터)은 독립 값(질량)에 대해 일정하다고 간주되므로 그래디언트는 0입니다.

In [ ]:
m1 = AutoDiffXd(2.0, [1.0, 0.0])
body1_ad.SetMass(context_ad, m1)

m2 = AutoDiffXd(0.5, [0.0, 1.0])
body2_ad.SetMass(context_ad, m2)

각 body $i$에 대해서 z-축 평행 이동 방향의 일반화된 힘은 단순히 $(-m_i \cdot g)$이며, 도함수는 $(-g)$입니다.

In [ ]:
def get_z_component(plant, body, v):
    assert body.is_floating()
    # Floating-base velocity dofs are organized as [angular velocity; translation velocity].
    v_start = body.floating_velocities_start_in_v()
    nv_pose = 6
    rxyz_txyz = v[v_start:v_start + nv_pose]
    assert len(rxyz_txyz) == nv_pose
    txyz = rxyz_txyz[-3:]
    z = txyz[2]
    return z

In [ ]:
@np.vectorize
def ad_to_string(x):
    # Formats an array of AutoDiffXd elements to a string.
    # Note that this implementation is for a scalar, but we use `np.vectorize` to
    # effectively convert our array to `ndarray` of strings.
    return f"AutoDiffXd({x.value()}, derivatives={x.derivatives()})"

In [ ]:
tau_g = plant_ad.CalcGravityGeneralizedForces(context_ad)
tau_g_z1 = get_z_component(plant_ad, body1_ad, tau_g)
tau_g_z2 = get_z_component(plant_ad, body2_ad, tau_g)
print(ad_to_string(tau_g_z1))
print(ad_to_string(tau_g_z2))